# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np # The Numpy numerical computing library
import pandas as pd # The Pandas data science library
import requests # The requests library for HTTP requests in Python
import xlsxwriter # The XlsxWriter libarary for 
import math # The Python math module
from scipy import stats # The SciPy stats module

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2039968454540,
 'week52high': 150.48,
 'week52low': 53.14,
 'week52highSplitAdjustOnly': 148.8,
 'week52lowSplitAdjustOnly': 54.4,
 'week52change': 0.8081628138056696,
 'sharesOutstanding': 17263319612,
 'float': 0,
 'avg10Volume': 115931248,
 'avg30Volume': 101946638,
 'day200MovingAvg': 126.81,
 'day50MovingAvg': 138.62,
 'employees': 149776,
 'ttmEPS': 3.82,
 'ttmDividendRate': 0.8426389566964837,
 'dividendYield': 0.006943049339748287,
 'nextDividendDate': '0',
 'exDividendDate': '2021-01-29',
 'nextEarningsDate': '2021-01-14',
 'peRatio': 33.15599103990524,
 'beta': 1.1495581629557756,
 'maxChangePercent': 45.55282845442878,
 'year5ChangePercent': 4.442870469134239,
 'year2ChangePercent': 1.9231297586983995,
 'year1ChangePercent': 0.6972734289744307,
 'ytdChangePercent': -0.09102270564130073,
 'month6ChangePercent': -0.027627335533130282,
 'month3ChangePercent': 0.04551018991831845,
 'month1ChangePercent': -0.1558141975779594,
 'day30Chan

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [4]:
data['year1ChangePercent']

0.6972734289744307

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,122.700,0.597649,N/A
1,AAL,21.161,-0.0984346,N/A
2,AAP,169.000,0.181783,N/A
3,AAPL,129.387,0.702096,N/A
4,ABBV,112.160,0.270613,N/A
...,...,...,...,...
500,YUM,107.820,0.0936324,N/A
501,ZBH,171.890,0.110572,N/A
502,ZBRA,522.880,1.26522,N/A
503,ZION,56.130,0.351739,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [7]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:51]
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,FCX,35.350,2.37054,N/A
1,CARR,38.140,2.10063,N/A
2,VIAC,66.020,1.62054,N/A
3,ALGN,581.820,1.44039,N/A
4,PYPL,260.300,1.37238,N/A
5,PWR,88.010,1.2849,N/A
6,LB,58.075,1.2777,N/A
7,ZBRA,522.880,1.26522,N/A
8,SIVB,524.253,1.25767,N/A
9,URI,305.380,1.20384,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()
print(portfolio_size)

Enter the value of your portfolio:10000
10000


In [9]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,FCX,35.350,2.37054,5
1,CARR,38.140,2.10063,5
2,VIAC,66.020,1.62054,2
3,ALGN,581.820,1.44039,0
4,PYPL,260.300,1.37238,0
5,PWR,88.010,1.2849,2
6,LB,58.075,1.2777,3
7,ZBRA,522.880,1.26522,0
8,SIVB,524.253,1.25767,0
9,URI,305.380,1.20384,0


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [10]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   'N/A',
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month6ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month3ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month1ChangePercent'],
                                                   'N/A',
                                                   'N/A'
                                                   ], 
                                                  index = hqm_columns), 
                                        ignore_index = True)
        
hqm_dataframe.columns

Index(['Ticker', 'Price', 'Number of Shares to Buy', 'One-Year Price Return',
       'One-Year Return Percentile', 'Six-Month Price Return',
       'Six-Month Return Percentile', 'Three-Month Price Return',
       'Three-Month Return Percentile', 'One-Month Price Return',
       'One-Month Return Percentile', 'HQM Score'],
      dtype='object')

## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [11]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        if hqm_dataframe.loc[row, change_col] == None:
            hqm_dataframe.loc[row, change_col] = 0.0


for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        hqm_dataframe.loc[row, percentile_col] = stats.percentileofscore(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])
        
# Print each percentile score to make sure it was calculated properly
for time_period in time_periods:
    print(hqm_dataframe[f'{time_period} Return Percentile'])

#Print the entire DataFrame    
hqm_dataframe

0      88.5149
1      12.8713
2      51.4851
3      92.4752
4      61.5842
        ...   
500    36.6337
501    39.0099
502    99.0099
503    71.2871
504     49.505
Name: One-Year Return Percentile, Length: 505, dtype: object
0       59.604
1      87.9208
2      26.1386
3      15.2475
4      47.9208
        ...   
500    37.4257
501    52.4752
502    94.8515
503     90.495
504    17.6238
Name: Six-Month Return Percentile, Length: 505, dtype: object
0       49.703
1      95.6436
2      52.4752
3      43.3663
4      42.9703
        ...   
500    27.1287
501    57.2277
502    93.6634
503    92.8713
504    24.3564
Name: Three-Month Return Percentile, Length: 505, dtype: object
0       30.297
1       99.604
2      45.5446
3      1.38614
4       30.099
        ...   
500    41.7822
501    54.8515
502    95.0495
503    89.1089
504     29.505
Name: One-Month Return Percentile, Length: 505, dtype: object


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,124.48,N/A,0.599222,88.5149,0.248918,59.604,0.0720136,49.703,-0.0274904,30.297,N/A
1,AAL,21.60,N/A,-0.101706,12.8713,0.612218,87.9208,0.396711,95.6436,0.36552,99.604,N/A
2,AAP,168.83,N/A,0.18487,51.4851,0.033799,26.1386,0.0796607,52.4752,0.00655066,45.5446,N/A
3,AAPL,125.43,N/A,0.720258,92.4752,-0.0282897,15.2475,0.0462596,43.3663,-0.156897,1.38614,N/A
4,ABBV,111.96,N/A,0.275241,61.5842,0.173077,47.9208,0.0440973,42.9703,-0.0289678,30.099,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,109.63,N/A,0.0920969,36.6337,0.0973719,37.4257,-0.0138155,27.1287,-0.000574709,41.7822,N/A
501,ZBH,168.22,N/A,0.110432,39.0099,0.200488,52.4752,0.101922,57.2277,0.0335645,54.8515,N/A
502,ZBRA,507.47,N/A,1.31238,99.0099,0.806327,94.8515,0.351141,93.6634,0.239549,95.0495,N/A
503,ZION,53.72,N/A,0.338451,71.2871,0.6775,90.495,0.342914,92.8713,0.169313,89.1089,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [12]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [13]:
hqm_dataframe.sort_values(by = 'HQM Score', ascending = False)
hqm_dataframe = hqm_dataframe[:51]

## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [14]:
portfolio_input()

Enter the value of your portfolio:10000


In [15]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe['Ticker'])-1):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])
hqm_dataframe

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,124.480,1,0.599222,88.5149,0.248918,59.604,0.0720136,49.703,-0.0274904,30.297,57.0297
1,AAL,21.600,9,-0.101706,12.8713,0.612218,87.9208,0.396711,95.6436,0.36552,99.604,74.0099
2,AAP,168.830,1,0.18487,51.4851,0.033799,26.1386,0.0796607,52.4752,0.00655066,45.5446,43.9109
3,AAPL,125.430,1,0.720258,92.4752,-0.0282897,15.2475,0.0462596,43.3663,-0.156897,1.38614,38.1188
4,ABBV,111.960,1,0.275241,61.5842,0.173077,47.9208,0.0440973,42.9703,-0.0289678,30.099,45.6436
5,ABC,101.922,1,0.126937,42.5743,0.0553375,29.3069,0.0016974,32.0792,-0.0344487,28.7129,33.1683
6,ABMD,338.000,0,1.06121,97.2277,0.0315231,25.9406,0.222883,79.2079,-0.082487,12.6733,53.7624
7,ABT,125.360,1,0.525716,84.3564,0.201575,52.6733,0.153192,69.3069,0.0598804,63.1683,67.3762
8,ACN,258.630,0,0.310432,67.9208,0.0748059,33.6634,0.0218032,39.0099,0.00719339,46.5347,46.7822
9,ADBE,463.065,0,0.324313,69.3069,-0.0539449,11.8812,-0.0235714,24.1584,-0.0308352,29.703,33.7624


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [16]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [17]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [18]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [19]:
writer.save()